In [9]:
import tensorflow as tf
import numpy as np
import shutil
import os
from simple_classifier import generate_bag_of_words, create_bag_of_word
import nltk
nltk.download('stopwords')
import csv

[nltk_data] Downloading package stopwords to /home/hdev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/hdev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
def read_files(filename):
    training_list = []
    label_list = []
    file = open(filename, "r")
    reader = csv.reader(file, delimiter=';')
    for tweet, author in reader:
        training_list.append(tweet)
        label_list.append(author)
    file.close()
    
    return {'x' : training_list, 'label' : label_list}

In [37]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]


In [49]:


class DeepNeuralNetwork:
    # Reference : 
    # https://www.tensorflow.org/get_started/mnist/beginners
    # http://web.stanford.edu/class/cs20si/lectures/notes_05.pdf
 
    def __init__(self, train_set, test_set, nb_classes, name = 'DNN'):
        
        self.train_set = train_set
        self.test_set = test_set
        self.nb_classes = nb_classes
        
        self.is_running = False
        self.model_name = name

        
        # TF saver to save regularly our progress
        self.global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name = 'global_step')
        
        self.saver =  tf.train.Saver()

    def create_model(self, hidden_layers):
        # At least one hidden layer !
        # Create the structure of the deep neural network
       
        input_layer_size = len(self.train_set['x'][0]) 
        output_layer_size = self.nb_classes

        self.x = tf.placeholder(tf.float32, [None, input_layer_size])
        self.y_ = tf.placeholder(tf.float32, [None, output_layer_size])
        
        self.hidden_layers = hidden_layers 

        self.weights = []
        self.layers = []
        self.bias = []
        
        W = tf.Variable(tf.random_normal([input_layer_size, self.hidden_layers[0]], stddev=0.01))
        self.weights.append(W) 
        b = tf.Variable(tf.zeros([self.hidden_layers[0]]))
        self.bias.append(b)
        y = tf.nn.sigmoid(tf.matmul(self.x, W)+b)
        self.layers.append(y)
        
        for i in range(len(self.hidden_layers)-1):
            W = tf.Variable(tf.random_normal([self.hidden_layers[i], self.hidden_layers[i+1]], stddev=0.1))
            self.weights.append(W) 
            b = tf.Variable(tf.zeros([self.hidden_layers[i+1]]))
            self.bias.append(b)
            y = tf.nn.sigmoid(tf.matmul(self.layers[i], W)+b)
            self.layers.append(y)

        W = tf.Variable(tf.random_normal([self.hidden_layers[-1], output_layer_size], stddev=0.1))
        self.weights.append(W)
        b = tf.Variable(tf.zeros([output_layer_size]))
        self.bias.append(b)
        
        self.y = tf.matmul(self.layers[-1], W)+b

        # Loss function
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = self.y_, logits = self.y))

        # Training accuracy
        correct_prediction = tf.equal(tf.argmax(self.y, 1), tf.argmax(self.y_, 1))
        self.training_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        # Optimizer
        self.train_step = tf.train.AdamOptimizer().minimize(self.loss, global_step = self.global_step)
    
    def train(self):
        
        if not self.is_running:
            self.run()
        n_epoch = 1
 
        """
        training_dataset = tf.data.Dataset.from_tensor_slices((self.train_set['x'], self.train_set['label']))    
        training_dataset = training_dataset.repeat(n_epoch)
        training_dataset = training_dataset.shuffle(buffer_size=10000)
        training_dataset = training_dataset.batch(32)
        
        iterator = training_dataset.make_one_shot_iterator()

        # self.sess.run(iterator.initializer)
  
        losses = []
        total_loss = 0
        step = 0
        while True:
            step += 1
            try:
                next_element = iterator.get_next()
                print(next_element)
                print(next_element[0])
                loss_batch, _, summary = self.sess.run([self.loss, self.train_step, self.summary_op], feed_dict={self.x:next_element[0], self.y_:next_element[1]})
                self.writer.add_summary(summary, global_step = step)
                total_loss += loss_batch    
                if step % 1000 == 0 :
                    losses.append(total_loss)
                    print(total_loss / 1000)
                    total_loss = 0
            except tf.errors.OutOfRangeError:
                break
        """
        # Generate batches
        batches = batch_iter(
            list(zip(self.train_set['x'], self.train_set['label'])), 32, n_epoch)
        
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            print(x_batch)
            print(y_batch)
            loss_batch, _, summary = self.sess.run([self.loss, self.train_step, self.summary_op], feed_dict={self.x : list(x_batch), self.y_ : list(y_batch)})
            self.writer.add_summary(summary, global_step = step)
            total_loss += loss_batch    
            if step % 1000 == 0 :
                losses.append(total_loss)
                print(total_loss / 1000)
                total_loss = 0
        
        """
        for batch in batches :
            print(batch)
            batch_xs, batch_ys = tf.next_batch(30,  self.train_set)
            loss_batch, _, summary = self.sess.run([self.loss, self.train_step, self.summary_op], feed_dict={self.x:batch_xs, self.y_:batch_ys})
            self.writer.add_summary(summary, global_step = step)
            total_loss += loss_batch    
            if step % 1000 == 0 :
                losses.append(total_loss)
                print(total_loss / 1000)
                total_loss = 0
        """
            

    def evaluate(self, x):
        if not self.is_running:
            self.run()

        return self.sess.run(tf.argmax(self.y, 1), feed_dict= {self.x: x})  


    def test(self):   

        correct_prediction = tf.equal(tf.argmax(self.y, 1), tf.argmax(self.y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        return self.sess.run(accuracy, feed_dict={self.x: self.test_set['x'], self.y_ : self.test_set['label']}) 
        
    
        
    def run(self):
        
        self.sess = tf.InteractiveSession()
        if os.path.exists('.graphs'):  
            shutil.rmtree('.graphs')
        
        self.writer = tf.summary.FileWriter('.graphs', self.sess.graph)    
        self.create_summaries()

        self.is_running = True
        tf.global_variables_initializer().run()
    
    def close(self):
        self.writer.close()
        self.sess.close()
        self.is_running = False

    def create_summaries(self):
        with tf.name_scope("summaries"):
            tf.summary.scalar("loss", self.loss)
            self.summary_op = tf.summary.merge_all()




In [45]:
train_set_raw = read_files('training.csv')
test_set_raw = read_files('test.csv')


train_set_features , word_dict = generate_bag_of_words(train_set_raw['x'], train_set_raw['label'])
test_set_features = create_bag_of_word(test_set_raw['x'], word_dict, False)


train_set = {'x' : train_set_features, 'label' : [np.array([float(s == 'Trump'),1-int(s == 'Trump')]) for s in train_set_raw['label']]}
test_set = {'x' : test_set_features, 'label' : [np.array([float(s == 'Trump'),1-int(s == 'Trump')]) for s in test_set_raw['label']]}

In [50]:
# example : 


DNN = DeepNeuralNetwork(train_set, test_set, 2, name='2_layers_perceptron')
DNN.create_model([1])
DNN.run()
print(DNN.sess.run(DNN.bias[0], feed_dict= {DNN.x: [test_set['x'][0]]}))
DNN.train()
print('accuracy : ' + str(DNN.test()))
print(DNN.evaluate(test_set['x']))
DNN.close()

[ 0.]
(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0,

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,146]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[?,146], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder', defined at:
  File "/home/hdev/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/hdev/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hdev/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hdev/anaconda3/envs/tensorflow/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/hdev/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home